# **Data Collection Notebook**

## Objectives

* Fetch data from Kaggle and save as raw data.
* Prepare data for future processes.

## Inputs

* Kaggle JSON file - authentification token for dataset access. 

## Outputs

* Generate Dataset: inputs/datasets/cherry_dataset

## Additional Comments

* Python 3.8.18 used as kernal during runtime.



---

Initial plan EDIT:

1:
Import packages
Install Kaggle %pip install kaggle==1.5.12

Get data from Kaggle
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json
KaggleDatasetPath = "codeinstitute/malaria-cell-classification" EDIT
DestinationFolder = "inputs/malaria_dataset"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}
unzip in destination
remove zip file

2:
DATA CLEANING
Remove any non image files see walkthgouh1
Split train validation test sets according to conventions 
Conventionally,
* The training set is divided into a 0.70 ratio of data.
* The validation set is divided into a 0.10 ratio of data.
* The test set is divided into a 0.20 ratio of data.

In [1]:
# TODO remove later

# import os
# import numpy as np
# import pandas as pd
# from PIL import Image
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.model_selection import train_test_split
# from joblib import dump, load
# import zipfile
# from typing import Tuple, List, Dict
# import logging

# class LeafImageProcessor:
#     def __init__(self, kaggle_dataset: str, destination_folder: str):
#         """
#         Initialize the image processor with dataset paths
        
#         Args:
#             kaggle_dataset: Kaggle dataset path
#             destination_folder: Local destination for dataset
#         """
#         self.kaggle_dataset = kaggle_dataset
#         self.destination_folder = destination_folder
#         self.setup_logging()
        
#     def setup_logging(self):
#         """Configure logging for the pipeline"""
#         logging.basicConfig(
#             level=logging.INFO,
#             format='%(asctime)s - %(levelname)s - %(message)s',
#             handlers=[
#                 logging.FileHandler('image_processing.log'),
#                 logging.StreamHandler()
#             ]
#         )
        
#     def load_kaggle_data(self) -> None:
#         """Download and extract Kaggle dataset"""
#         try:
#             # Ensure Kaggle credentials are properly set
#             os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
#             os.chmod('kaggle.json', 0o600)
            
#             # Download dataset
#             os.system(f'kaggle datasets download -d {self.kaggle_dataset} -p {self.destination_folder}')
            
#             # Extract dataset
#             zip_files = [f for f in os.listdir(self.destination_folder) if f.endswith('.zip')]
#             for zip_file in zip_files:
#                 zip_path = os.path.join(self.destination_folder, zip_file)
#                 with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#                     zip_ref.extractall(self.destination_folder)
#                 os.remove(zip_path)
                
#             logging.info("Dataset successfully downloaded and extracted")
            
#         except Exception as e:
#             logging.error(f"Error in data loading: {str(e)}")
#             raise
            
#     def clean_data(self) -> Tuple[List[str], List[str]]:
#         """
#         Clean data directory and split into train/val/test sets
        
#         Returns:
#             Tuple of training and validation file paths
#         """
#         try:
#             # Get all image files
#             image_files = []
#             for root, _, files in os.walk(self.destination_folder):
#                 for file in files:
#                     if file.lower().endswith(('.png', '.jpg', '.jpeg')):
#                         image_files.append(os.path.join(root, file))
                        
#             # Remove non-image files
#             for root, _, files in os.walk(self.destination_folder):
#                 for file in files:
#                     if not file.lower().endswith(('.png', '.jpg', '.jpeg')):
#                         os.remove(os.path.join(root, file))
                        
#             # Split data
#             train_files, test_files = train_test_split(image_files, test_size=0.2, random_state=42)
#             train_files, val_files = train_test_split(train_files, test_size=0.125, random_state=42)  # 0.125 of 80% = 10% of total
            
#             logging.info(f"Data split complete: {len(train_files)} train, {len(val_files)} val, {len(test_files)} test")
#             return train_files, val_files
            
#         except Exception as e:
#             logging.error(f"Error in data cleaning: {str(e)}")
#             raise
            
#     def analyze_image_statistics(self, image_files: List[str]) -> Dict:
#         """
#         Compute image statistics
        
#         Args:
#             image_files: List of image file paths
            
#         Returns:
#             Dictionary of image statistics
#         """
#         try:
#             sizes = []
#             for img_path in image_files:
#                 with Image.open(img_path) as img:
#                     sizes.append(img.size)
                    
#             sizes = np.array(sizes)
#             stats = {
#                 'mean_width': np.mean(sizes[:, 0]),
#                 'mean_height': np.mean(sizes[:, 1]),
#                 'std_width': np.std(sizes[:, 0]),
#                 'std_height': np.std(sizes[:, 1])
#             }
            
#             logging.info(f"Image statistics computed: {stats}")
#             return stats
            
#         except Exception as e:
#             logging.error(f"Error in computing image statistics: {str(e)}")
#             raise
            
#     def create_visualizations(self, train_files: List[str], save_dir: str = 'outputs/visualizations'):
#         """
#         Create and save visualizations
        
#         Args:
#             train_files: List of training image files
#             save_dir: Directory to save visualizations
#         """
#         try:
#             os.makedirs(save_dir, exist_ok=True)
            
#             # Average image per class
#             healthy_imgs = [img for img in train_files if 'healthy' in img.lower()]
#             infected_imgs = [img for img in train_files if 'powdery_mildew' in img.lower()]
            
#             def compute_average_image(file_list):
#                 images = []
#                 target_size = (100, 100)
#                 for f in file_list[:100]:  # Limit to prevent memory issues
#                     with Image.open(f) as img:
#                         img_resized = img.resize(target_size)
#                         images.append(np.array(img_resized))
#                 return np.mean(images, axis=0)
            
#             # Create visualization plots
#             avg_healthy = compute_average_image(healthy_imgs)
#             avg_infected = compute_average_image(infected_imgs)
#             difference = avg_infected - avg_healthy
            
#             # Plot and save
#             fig, axes = plt.subplots(1, 3, figsize=(15, 5))
#             axes[0].imshow(avg_healthy)
#             axes[0].set_title('Average Healthy Leaf')
#             axes[1].imshow(avg_infected)
#             axes[1].set_title('Average Infected Leaf')
#             axes[2].imshow(difference)
#             axes[2].set_title('Difference')
#             plt.savefig(os.path.join(save_dir, 'average_images.png'))
            
#             logging.info("Visualizations created and saved")
            
#         except Exception as e:
#             logging.error(f"Error in creating visualizations: {str(e)}")
#             raise
            
#     def save_artifacts(self, stats: Dict, save_dir: str = 'outputs/artifacts'):
#         """
#         Save processing artifacts
        
#         Args:
#             stats: Dictionary of computed statistics
#             save_dir: Directory to save artifacts
#         """
#         try:
#             os.makedirs(save_dir, exist_ok=True)
            
#             # Save statistics
#             dump(stats, os.path.join(save_dir, 'image_statistics.pkl'))
            
#             # Save summary as text
#             with open(os.path.join(save_dir, 'processing_summary.txt'), 'w') as f:
#                 f.write(f"Image Processing Summary\n{'='*20}\n")
#                 for key, value in stats.items():
#                     f.write(f"{key}: {value}\n")
                    
#             logging.info(f"Artifacts saved to {save_dir}")
            
#         except Exception as e:
#             logging.error(f"Error in saving artifacts: {str(e)}")
#             raise

# # Example usage
# if __name__ == "__main__":
#     processor = LeafImageProcessor(
#         kaggle_dataset="codeinstitute/cherry-leaves",  # Update with correct dataset
#         destination_folder="inputs/cherry_leaves"
#     )
    
#     # Execute pipeline
#     processor.load_kaggle_data()
#     train_files, val_files = processor.clean_data()
#     stats = processor.analyze_image_statistics(train_files)
#     processor.create_visualizations(train_files)
#     processor.save_artifacts(stats)

TODO remove later

Possible enhancements to initial workflow:


Data Loading:

Added error handling for Kaggle API
Included validation of downloaded files
Added logging for tracking download progress


Data Cleaning:

Automated removal of non-image files
Implemented proper train/val/test split with fixed random seed
Added validation of image integrity


Data Visualization:

Enhanced image statistics computation
Added memory-efficient processing for large datasets
Included proper error handling for corrupt images


Additional Features:

Logging system for tracking processing steps
Artifact management for saving results
Type hints for better code maintainability



Key improvements over your initial plan:

Error Handling:

Robust error checking at each step
Graceful handling of corrupt images


Memory Management:

Batch processing for large datasets
Efficient image loading and processing


Documentation:

Clear logging of each step
Comprehensive statistics saved as artifacts


Extensibility:

Easy to add new visualization types
Modular design for adding new processing steps

# Set up notebook workspace

* change working directory to parent for requirements access
* confirm python version for continuity (3.8.18)
* install requirements

In [2]:
# Notebooks saved in subdirectory. Root access required for installing required packages.
import os

# get current directory
current_dir = os.getcwd() 
current_dir

'/workspace/cherry-ML/jupyter_notebooks'

In [3]:
os.chdir(os.path.dirname(current_dir))  # change to parent dir
print("You set a new current directory")

You set a new current directory


In [4]:
# confirm the new current directory
current_dir = os.getcwd()
current_dir

'/workspace/cherry-ML'

### Confirm Python Version

In [5]:
# confirm python version is 3.8.18 for continuity
! python --version

Python 3.8.18


### Install packages

In [6]:
!pwd
%pip install -r requirements.txt


/workspace/cherry-ML
Note: you may need to restart the kernel to use updated packages.


# Collect Data via Kaggle API

Install Kaggle

In [7]:
# confirm kaggle installed if not already
%pip install kaggle==1.5.12

Note: you may need to restart the kernel to use updated packages.


Allow Kaggle configuration using auth JSON setting config dir to current dir.

In [8]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

Set Kaggle Dataset and download it.

In [9]:
KaggleDatasetPath = "codeinstitute/cherry-leaves"
DestinationFolder = "inputs/cherry-leaves-dataset"  # creates new dir/dir
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

 95%|███████████████████████████████████▉  | 52.0M/55.0M [00:02<00:00, 18.0MB/s]
100%|██████████████████████████████████████| 55.0M/55.0M [00:02<00:00, 20.0MB/s]


Unzip downloaded dataset file

In [10]:
import zipfile

try:
    with zipfile.ZipFile(DestinationFolder + '/cherry-leaves.zip', 'r') as zip_ref:
        zip_ref.extractall(DestinationFolder)
except Exception as e:
    print(e)


Delete redundant zip file

In [11]:
os.remove(DestinationFolder + '/cherry-leaves.zip')

---

# Data Preparation

# Data Cleaning

* remove any non-image files

In [12]:
def remove_non_image_files(my_data_dir):
    print('Removing non image files...\n')
    image_extension = ('.png', '.jpg', 'jpeg')
    folders = os.listdir(my_data_dir)
    for folder in folders:
        files = os.listdir(f'{my_data_dir}/{folder}')
        # print files
        non_image = []
        image_count = []
        for given_file in files:
            try:
                if not given_file.lower().endswith(image_extension):
                    file_location = f'{my_data_dir}/{folder}/{given_file}'
                    os.remove(file_location) # remove non image file
                    non_image.append(1)
                else:
                    image_count.append(1)
                    pass
            except Exception as e:
                print(e)

        print(f'Folder: {folder} has - {len(image_count)} image files')
        print(f'Folder: {folder} has - {len(non_image)} non image files, which have been removed')

In [13]:
remove_non_image_files('inputs/cherry-leaves-dataset/cherry-leaves')

Removing non image files...

[Errno 21] Is a directory: 'inputs/cherry-leaves-dataset/cherry-leaves/test/healthy'
[Errno 21] Is a directory: 'inputs/cherry-leaves-dataset/cherry-leaves/test/powdery_mildew'
Folder: test has - 0 image files
Folder: test has - 0 non image files, which have been removed
[Errno 21] Is a directory: 'inputs/cherry-leaves-dataset/cherry-leaves/train/healthy'
[Errno 21] Is a directory: 'inputs/cherry-leaves-dataset/cherry-leaves/train/powdery_mildew'
Folder: train has - 0 image files
Folder: train has - 0 non image files, which have been removed
[Errno 21] Is a directory: 'inputs/cherry-leaves-dataset/cherry-leaves/validation/healthy'
[Errno 21] Is a directory: 'inputs/cherry-leaves-dataset/cherry-leaves/validation/powdery_mildew'
Folder: validation has - 0 image files
Folder: validation has - 0 non image files, which have been removed
Folder: healthy has - 2104 image files
Folder: healthy has - 0 non image files, which have been removed
Folder: powdery_mildew 

# Split train validation test sets

In [14]:
import os
import shutil
import random
import joblib

def split_train_validation_test_images(my_data_dir, train_set_ratio, validation_set_ratio, test_set_ratio):

    # confirm ratios total 1.0
    if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
        print('Ratios should total 1.0.')
        print('You entered:\n')
        print(f'Train radio: {train_set_ratio}')
        print(f'Validation radio: {validation_set_ratio}')
        print(f'Test radio: {test_set_ratio}')
        return 

    # get classes labels
    labels = os.listdir(my_data_dir)  # expect only folder name
    if 'test' in labels:
        pass
    else:
        try:
            # create train, test folders with classes labels sub-folder
            for folder in ['train', 'validation', 'test']:
                for label in labels:
                    os.makedirs(name=f'{my_data_dir}/{folder}/{label}')
            
            for label in labels:

                files = os.listdir(f'{my_data_dir}/{label}')
                random.shuffle(files)

                train_set_files_qty = int(len(files) * train_set_ratio)
                validation_set_files_qty = int(len(files) * validation_set_ratio)

                count = 1
                for file_name in files:
                    if count <= train_set_files_qty:
                        # move given file to train set
                        shutil.move(f'{my_data_dir}/{label}/{file_name}',
                                    f'{my_data_dir}/train/{label}/{file_name}')
                    elif count <= (train_set_files_qty + validation_set_files_qty):
                        # move a given file to the validation set
                        shutil.move(f'{my_data_dir}/{label}/{file_name}',
                                    f'{my_data_dir}/validation/{label}/{file_name}')
                    else:
                        # move given file to test set
                        shutil.move(f'{my_data_dir}/{label}/{file_name}',
                                    f'{my_data_dir}/test/{label}/{file_name}')
                    
                    count += 1

                os.rmdir(f'{my_data_dir}/{label}')
            
        except Exception as e:
            print(e)
    print('Done!')





Conventional ratios will be followed here:
* The training set is divided into a 0.70 ratio of data.
* The validation set is divided into a 0.10 ratio of data.
* The test set is divided into a 0.20 ratio of data.

In [18]:
split_train_validation_test_images(
    my_data_dir='inputs/cherry-leaves-dataset/cherry-leaves',
    train_set_ratio=0.7,
    validation_set_ratio=0.1,
    test_set_ratio=0.2
)

Done!


---

# Prepare for push to repository

* Ensure the downloaded files are not added to repo by including in .gitinore file (they can be redownloaded in the workspace if neccessary)

In [16]:
!echo "/inputs/cherry-leaves-dataset/cherry-leaves/" >> .gitignore  # add if doesn't already exists (bash)

Confirm included in .gitignore before pushing to repo

In [17]:
cat .gitignore

core.Microsoft*
core.mongo*
core.python*
env.py
__pycache__/
*.py[cod]
node_modules/
.github/
cloudinary_python.txt
kaggle.json-e 
/inputs/cherry-leaves-dataset/cherry-leaves/
/inputs/cherry-leaves-dataset/cherry-leaves/


Now commit to repository.

All files are now prepared for data exploration.
Only images are in the dataset according to defined file types. 